# ĐỒ ÁN CUỐI KỲ 

Họ tên: Huỳnh Ngọc Quân

MSSV: 1712689

## Import

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Lấy dữ liệu

Dữ liệu trong đồ án cuối kỳ này bao gồm: 
- Dữ liệu về các unicorns trên thế giới tính tới thời 29/1/2020 [ở đây](https://www.kaggle.com/narmelan/global-unicorns-2019) ***CB-Insights_Global-Unicorns_2019.csv***
- Dữ liệu các start up ở Ấn Độ trong các lĩnh vực và thông tin về các nhà đầu tư [ở đây](https://www.kaggle.com/sudalairajkumar/indian-startup-funding) ***startup_funding.csv***
- Dữ liệu các nhà đầu tư cho start up trên thế giới do unicorn-nest tổng hợp tính tới 4/1/2020 [ở đây](https://www.kaggle.com/iwanenko/unicorn-nest-startup-fundraising-dataset) ***funds.csv***

Chúng ta sẽ phân tích sự phân bố unicorns trên thế giới, các nhà đầu tư mạnh trong các lĩnh vực và xu hướng đầu tư hiện nay của thế giới. 

Sau đó ta sẽ phân tích dữ liệu các start up ở Ấn Độ để tìm ra được một vài kết luận thú vị.

# 1. Khám phá dữ liệu

## 1.1 Đọc dữ liệu từ file csv 

In [2]:
unicorns_df = pd.read_csv("CB-Insights_Global-Unicorns_2019.csv")
startup_df = pd.read_csv("startup_funding.csv")
investor_df = pd.read_csv("funds.csv")

## 1. 2 Tổng quan về dữ liệu

### 1.2.1 Dữ liệu unicorn

#### 1.2.1.1 Ý nghĩa dữ liệu

In [3]:
unicorns_df.head()

,Company,Valuation ($B),Date Joined,Country,Category,Select Investors
0,Toutiao (Bytedance),$75.00,4/7/2017,China,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S..."
1,Didi Chuxing,$56.00,12/31/2014,China,Auto & transportation,"Matrix Partners, Tiger Global Management, Soft..."
2,JUUL Labs,$50.00,12/20/2017,United States,Consumer & retail,Tiger Global Management
3,SpaceX,$33.30,12/1/2012,United States,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
4,Stripe,$35.25,1/23/2014,United States,Fintech,"Khosla Ventures, Lowercase Capital, capitalG"


In [4]:
unicorns_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449 entries, 0 to 448
Data columns (total 6 columns):
Company             449 non-null object
Valuation ($B)      449 non-null object
Date Joined         449 non-null object
Country             449 non-null object
Category            449 non-null object
Select Investors    448 non-null object
dtypes: object(6)
memory usage: 21.2+ KB


In [5]:
unicorns_df.nunique()

Company             449
Valuation ($B)      106
Date Joined         383
Country              27
Category             16
Select Investors    447
dtype: int64

In [6]:
unicorns_df.shape

(449, 6)

In [7]:
print("NaN")
print(unicorns_df.isnull().sum())

NaN
Company             0
Valuation ($B)      0
Date Joined         0
Country             0
Category            0
Select Investors    1
dtype: int64


#### Ý nghĩa các cột dữ liệu:

- **Company**: Tên công ty (start up)
- **Valuation**: Định giá công ty (tỉ USD)
- **Date Joined**: Ngày được công nhận là unicorn
- **Country**: Quốc gia
- **Category**: Lĩnh vực công ty
- **Select Investors**: Các nhà đầu tư

**Tổng quan về các dòng và cột dữ liệu**

Dữ liệu gồm 449 dòng và 6 cột:
- Chỉ có 1 dòng dữ liệu có dữ liệu NaN ở cột Select Investors.
- Có 449 unicorns được ghi nhận tính tới thời 29/1/2020.
- Có 27 quốc gia có unicorns.
- Có 16 lĩnh vực được quan tâm và trở thành unicorns.
- Cả 6 cột đều có kiểu dữ liệu là object, nhưng ta nhận thấy ở cột thứ 2 kiểu dữ liệu nên là numerical, và cột thứ 3 là datetime. Ta sẽ tiền xử lý ở bước sau.

### 1.2.1.2 Tiền xử lý 

**Cột Valuadation**

- Xóa ký tự \$ ở đầu và chuyển sang dạng numerical

In [8]:
unicorns_df = unicorns_df.replace('\$','', regex=True)
unicorns_df["Valuation ($B) "] = unicorns_df["Valuation ($B) "].astype(float)
unicorns_df.head()

,Company,Valuation ($B),Date Joined,Country,Category,Select Investors
0,Toutiao (Bytedance),75.00,4/7/2017,China,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S..."
1,Didi Chuxing,56.00,12/31/2014,China,Auto & transportation,"Matrix Partners, Tiger Global Management, Soft..."
2,JUUL Labs,50.00,12/20/2017,United States,Consumer & retail,Tiger Global Management
3,SpaceX,33.30,12/1/2012,United States,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
4,Stripe,35.25,1/23/2014,United States,Fintech,"Khosla Ventures, Lowercase Capital, capitalG"


**Cột Date Joined**

- Chuyển về dạng datetime 

In [9]:
unicorns_df['Date Joined'] =  pd.to_datetime(unicorns_df['Date Joined'])
unicorns_df.head()

,Company,Valuation ($B),Date Joined,Country,Category,Select Investors
0,Toutiao (Bytedance),75.00,2017-04-07,China,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S..."
1,Didi Chuxing,56.00,2014-12-31,China,Auto & transportation,"Matrix Partners, Tiger Global Management, Soft..."
2,JUUL Labs,50.00,2017-12-20,United States,Consumer & retail,Tiger Global Management
3,SpaceX,33.30,2012-12-01,United States,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
4,Stripe,35.25,2014-01-23,United States,Fintech,"Khosla Ventures, Lowercase Capital, capitalG"


**Dữ liệu sau khi tiền xử lý**

In [10]:
unicorns_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449 entries, 0 to 448
Data columns (total 6 columns):
Company             449 non-null object
Valuation ($B)      449 non-null float64
Date Joined         449 non-null datetime64[ns]
Country             449 non-null object
Category            449 non-null object
Select Investors    448 non-null object
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 21.2+ KB


In [ ]:
unicorns_df.head

### 1.2.2 Dữ liệu startup ở Ấn Độ

#### 1.2.2.1 Ý nghĩa dữ liệu

In [11]:
startup_df.head()

,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN


In [12]:
startup_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3044 entries, 0 to 3043
Data columns (total 10 columns):
Sr No                3044 non-null int64
Date dd/mm/yyyy      3044 non-null object
Startup Name         3044 non-null object
Industry Vertical    2873 non-null object
SubVertical          2108 non-null object
City  Location       2864 non-null object
Investors Name       3020 non-null object
InvestmentnType      3040 non-null object
Amount in USD        2084 non-null object
Remarks              419 non-null object
dtypes: int64(1), object(9)
memory usage: 237.9+ KB


In [13]:
startup_df.nunique()

Sr No                3044
Date dd/mm/yyyy      1035
Startup Name         2459
Industry Vertical     821
SubVertical          1942
City  Location        112
Investors Name       2412
InvestmentnType        55
Amount in USD         471
Remarks                72
dtype: int64

In [14]:
print("NaN")
print(startup_df.isnull().sum())

NaN
Sr No                   0
Date dd/mm/yyyy         0
Startup Name            0
Industry Vertical     171
SubVertical           936
City  Location        180
Investors Name         24
InvestmentnType         4
Amount in USD         960
Remarks              2625
dtype: int64


In [15]:
startup_df.shape

(3044, 10)

In [16]:
startup_df = startup_df.drop('Remarks', axis = 1)

#### Ý nghĩa các cột dữ liệu:

- **Date**: Ngày nhận vốn đầu tư phát triển
- **Startup Name**: Tên công ty (start up)
- **Industry Vertical**: Lĩnh vực chính của công ty
- **SubVertical**: Lĩnh vực phụ của công ty
- **City Location**: Thành phố của trụ sở công ty
- **Investors Name**: Tên nhà đầu tư
- **InvestmentnType**: Loại hình đầu tư
- **Amount in USD**: Số tiền đầu tư (theo USD)

#### 1.2.2.2 Tiền xử lý 

**Cột Date**
- Chuyển về dạng datetime

Tìm các ngày không hợp lệ trong cột Date dd/mm/yyyy và sửa lại thành các ngày hợp lệ.

In [17]:
# Lấy tất cả các giá trị cột Date dd/mm/yyyy
data = startup_df['Date dd/mm/yyyy'].values

# Đưa vào dataframe
df = pd.DataFrame({'Date': data})

# Thêm cột correct để đánh giá format đúng hay chưa
df['correct'] = pd.to_datetime(df['Date'],errors='coerce')

# Danh sách các giá trị không hợp lệ
errors = df.loc[df['correct'].isnull()]['Date'].unique().tolist()
errors

['05/072018', '01/07/015', '\\\\xc2\\\\xa010/7/2015', '22/01//2015']

In [18]:
startup_df['Date dd/mm/yyyy'].replace((['05/072018', '01/07/015','22/01//2015', '\\\\xc2\\\\xa010/7/2015']),
                         ('05/07/2018','01/07/2015','22/01/2015', '10/07/2015'), inplace=True)

In [19]:
startup_df['Date dd/mm/yyyy'] = pd.to_datetime(startup_df['Date dd/mm/yyyy'])

ValueError: ('Unknown string format:', '\\\\xc2\\\\xa010/7/2015')

**Cột Amount in USD**
- Chuyển về dạng float

Khi chuyển dữ liệu sang float, ta lần lượt gặp các lỗi về giá trị, chúng ta sẽ lần lượt xử lý các giá trị này để đưa chung về một format:

- Các chuỗi bắt đầu bằng \\\\xc2\\\\xa0 + giá trị số
- Các giá trị đặc biệt: undisclosed, unknown, Undisclosed, N/A chuyển thành NaN

In [ ]:
startup_df["Amount in USD"] = startup_df["Amount in USD"].apply(lambda x: str(str(x).replace(",","").replace("+", "")))

In [ ]:
startup_df = startup_df.replace('\\\\\\\\xc2\\\\\\\\xa0','', regex=True)

In [ ]:
startup_df['Amount in USD'].replace((['undisclosed', 'unknown','Undisclosed', 'N/A']),
                         ('NaN','NaN','NaN','NaN'), inplace=True)

In [ ]:
startup_df["Amount in USD"] = startup_df["Amount in USD"].apply(lambda x: float(x))

**Dữ liệu sau khi tiền xử lý**

In [ ]:
startup_df.info()

In [ ]:
startup_df.head()

### 1.2.3 Dữ liệu các nhà đầu tư

In [ ]:
investor_df.head()

In [ ]:
investor_df.info()

#### Xóa các cột dữ liệu không cần thiết
Dữ liệu có 28 cột, ta xóa bớt các cột thông tin về các nhà đầu tư như:

- Unicorn Nest website link
- Fund website
- Fund LinkedIn
- Fund Crunchbase

...

Và chỉ lấy các cột có thông tin phân tích đối với 2 tập dữ liệu còn lại:

- **Fund name**: Tên nhà đầu tư
- **Fund structure**: Phân biệt giữa 2 cấu trúc quỹ đầu tư mạo hiểm, và quỹ đầu tư mạo hiểm doanh nghiệp
- **Country**: Quốc gia
- **Some of TOP industries**: Các lĩnh vực đầu tư
- **One of the most outstanding startups**: Các start up nổi bật
- **Unicorn amount**: Số lượng Unicorn
- **Sucess / strategy diversity index**: Mức độ thành công

In [ ]:
investor_df = investor_df.drop(['Unicorn Nest website link', 'Fund website', 'Fund LinkedIn', 
                                'Fund Crunchbase', 'Email format for website', 'Coincidence of the geography of the fund and its TOP1 invest geofocus',
                                'Fund that most often invests in previous rounds',
                                'Fund that most often invests together', 'Fund that most often invests in following rounds',
                                'Fund novelty', 'Proximity to the year of peak activity', 'Amount of Key Persons', 
                                'Lead investments', 'Exits', 'Average rounds per year', 
                                'Average round size', 'Average startup valuation', 'Decision makers / strategy diversity index',
                                'Average multiplicator', 'Follow-on index', 'Readiness to take younger startups'
                               ], axis = 1)
investor_df.head()